## Project 2 - Applications of modal decomposition to reduced order modelling

In this mini project you will explore applications of POD and DMD to reduced order modelling. As both POD and DMD provide a low-rank basis on which high-dimensional fluid data can be projected, they provide an ideal way to reduce the complexity of the flow before a reduced order modelling technique is applied. The reduced order modelling technique we will consider is the Sparse Identification of Nonlinear Dynamics (SINDy). The project consists of 6 questions with a total of 50 points.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pysindy as ps
import scipy.sparse as sp
import helper_functions as helper_functions

For help in completing this project, the **helper_functions** Python module is included with this notebook. This module contains functions from the class notebook for performing modal decomposition, as well as new functions that aid the creation of plots for this project. For your first task, familiarise yourself with the functions in this module. 

**Problem 0** Add comments to the DMD routine outlining the main steps in the algorithm. For the purpose of your submission, paste the commented routine below. (5 marks)

## Part 0: Introduction to SINDy
[SINDy](https://pysindy.readthedocs.io/en/latest/examples/2_introduction_to_sindy/example.html) seeks to find governing equations from data. If measurements 
$$\mathbf{X}=
\begin{pmatrix}
x_0(t_0) & x_1(t_0) & \cdots & x_{m-1}(t_0) \\
x_0(t_1) & x_1(t_1) & \cdots & x_{m-1}(t_1) \\
\vdots & \vdots & & \vdots \\
x_0(t_{n-1}) & x_1(t_{n-1}) & \cdots & x_{m-1}(t_{n-1})
\end{pmatrix},
$$
are taken from a system ($m$ variables and measurements at $n$-points in time) then SINDy solves the regression problem
$$
\dot{\mathbf{X}}=\boldsymbol{\Theta}(\mathbf{X})\boldsymbol{\Xi},
$$
where the coefficients $\boldsymbol{\Xi}$ are constrained to be sparse and $\boldsymbol{\Theta}$ is a user chosen library of candidate terms. The most common choice for $\boldsymbol{\Theta}$ is a polynomial basis up to a specified degree and this is sufficient for completing the mini project. Let's illustrate this procedure on the following data.

In [ ]:
# Load the data
helper_functions.download_data()
data = np.load('data/lorentz.npz')
sol = data['sol']
t = data['t']

In [ ]:
# Plot the data
helper_functions.plot_ODE_data(sol)

You may have already guessed from the file name and 3D visualisation that this data was generated from the famous Lorentz equations
\begin{align}
\frac{\textrm{d} x}{\textrm{d} t} &=& \sigma(y-z),\\
\frac{\textrm{d} y}{\textrm{d} t} &=& x(\rho - z) - y,\\
\frac{\textrm{d} z}{\textrm{d} t} &=& xy-\beta z,
\end{align}
with $\sigma=10$, $\rho=28$, and $\beta=8/3$. Let's now confirm our suspicions by getting SINDy to fit the equations. To do this we can set up a SINDy model. We optionally tell it that the feature names are $x$, $y$, and $z$ for visualisation purposes. The Lorentz equations have a quadratic nonlinearity, but for the feature library $\boldsymbol{\Theta}$, we choose polynomials up to degree 3 which includes the possibility of cubic terms. Lastly for the optimiser we choose sequentially thresholded least squares with a threshold of $0.1$ (which controls the sparsity of the solution). With the model set up, we can fit the model to our observed solution and print it.

In [ ]:
model = ps.SINDy(feature_names=["x", "y", "z"], 
                 feature_library=ps.PolynomialLibrary(degree=3),
                 optimizer=ps.STLSQ(threshold=1e-1),
                 )
model.fit(sol, t=t, quiet=True)
model.print()

And that's it! SINDy has fit the equations and found coefficients very close to the ones we expected. By adjusting the threshold you can see that if it is too large, we do not recover all the terms, and if it is too small we gain additional terms in our model. Choosing the correct threshold and feature library is down to the user, and should be adjusted until the model is just simple enough to explain the results. With the model found we can also use SINDy to run it, and check if we get the desired behaviour.

In [ ]:
times = np.arange(0, 30, 0.01)
initial_condition = [0.01, 0.01, 0.01] 
x_sindy = model.simulate(initial_condition, times)
helper_functions.plot_ODE_data(x_sindy)

**Problem 1** For the three data-sets 'equation_A.npz', 'equation_B.npz', and 'equation_C.npz', plot the ODE data in phase space using and use SINDy to recover the governing equations that produced them. For each equation, explain your choice of threshold and polynomial library. Note, the *2D* ODE data is stored in exactly the same manner as for the Lorentz example. (10 marks)

## Part 1: SINDy for fluid-flow data


With the SINDy procedure illustrated, let's explore reduced order modelling for fluid flow data. Specifically, let's consider flow past a cylinder at $\textit{Re}=100$, which is a canonical fluid dynamics example often used to benchmark data-driven techniques. The vortex street seen in the wake of a cylinder arises from a Hopf Bifurcation at $\textit{Re}\approx47$. For Reynolds numbers below this there exists a steady solution to the governing equations. However, for Reynolds numbers greater than this critical value, instability gives rises to a periodic solution (the von Kármán vortex street). In the SINDy examples seen in this notebook so far, the data is already given in coordinates in which the governing equation is an ODE with sparse nonlinearities. However, fluid-flow data is inherently high dimensional and cannot easily be directly inputted to SINDy. For the rest of the mini project we will investigate how modal decomposition can be used to reduce the high dimensional flow data to a low-rank representation that SINDy can learn a model for.

The cylinder flow dataset contains data for flow past a cylinder obtained by a simulation performed using the immersed boundary technique on a staggered mesh. This means that the flow in the $x$-direction $u$ and flow in the $y$-direction $v$, are provided at different $x$ and $y$ coordinates. The arrays provide by the dataset are
1. `u`/ `v` : $x$/$y$ components of velocity with shape (number of snapshots, $x$ resolution, $y$ resolution)
2. `xu`/`xv` : $x$ coordinates for the $u$/$v$ fields.
3. `yu`/`yv` : $y$ coordinates for the $u$/$v$ fields.
4. `t` : Times at which the snapshots are obtained.
5. `lift` : Lift force on the cylinder at every timestep in the simulation (not the same as the timesteps where the flow snapshots are taken)
6. `lift_time` : Times at which the lift data is provided.

To illustrate this, below we load the data and plot the $u$ and $v$ components for the final snapshot.

In [ ]:
data = np.load('data/cylinder_flow_data.npz')

xu = data['xu']
yu = data['yu']
u = data['u']

xv = data['xv']
yv = data['yv']
v = data['v']
t = data['t']
lift  = data['lift']
lift_time  = data['lift_time']

# Plot the final snapshot
fig, ax = plt.subplots(2, 1, layout='tight')
helper_functions.plot_cylinder_data(xu, yu, u[-1], fig_ax=(fig, ax[0]))
helper_functions.plot_cylinder_data(xv, yv, v[-1], fig_ax=(fig, ax[1]))

# Plot the lift against time
plt.figure()
plt.plot(lift_time, lift)
plt.xlabel(r'$t$')
plt.ylabel(r'Lift')

Also provided, are mass matrices `mass_u` and `mass_v` contained in *cylinder_mass_u.npz* and *cylinder_mass_v.npz*, respectively. These allow for energy-based norms to be used with modal decomposition techniques. The are shaped to work with the **flattened** `u` and `v` arrays. To illustrate this, below we calculate the kinetic energy of the last snapshot.

In [ ]:
mass_u = sp.load_npz('data/cylinder_mass_u.npz')
mass_v = sp.load_npz('data/cylinder_mass_v.npz')

In [ ]:
u_last_snapshot_flattened = u[-1].flatten()
v_last_snapshot_flattened = v[-1].flatten()
KE = np.dot(u_last_snapshot_flattened, mass_u @ u_last_snapshot_flattened) + np.dot(v_last_snapshot_flattened, mass_v @ v_last_snapshot_flattened) 
print('KE =', KE)

Let's use SINDy to create a model for the evolution from the unstable, steady, base-state to the periodic solution. This is similar to one of the examples in the original SINDy [paper](https://www.pnas.org/doi/10.1073/pnas.1517384113). To this end, we first need to identify from data the steady base-state.

**Problem 2**: Using DMD, find the unstable steady base-state $\mathbf{q}_{\textrm{FP}}$. Compare this to the mean-flow of the limit cycle $\bar{\mathbf{q}}_{\textrm{LC}}$. What are the differences? Physically, why is the unstable base-state different from the mean-flow of the limit cycle? Why was DMD, rather than POD, the appopriate choice of modal decomposition technique? (10 marks)

This unstable fixed point will form the basis of our SINDy model. In order to represent the growth and saturation of the instability of our unstable fixed point, we will use three modes. Two of these will represent the periodic limit cycle dynamics, and the third will govern the unoscillatory shift of the mean flow from the unstable fixed point to the limit cycle mean-flow.

**Problem 3**: Find an *orthogonal* basis containing three vectors. One for the mean-shift mode $\mathbf{q}_0$, and two modes $\mathbf{q}_1$ and $\mathbf{q}_2$ representing the limit cycle dynamics. In other words, find $\mathbf{q}_0$, $\mathbf{q}_1$, $\mathbf{q}_2$, such that
$$
\mathbf{q}=\mathbf{q}_{\textrm{FP}} + a_0(t)\mathbf{q}_0 + a_1(t)\mathbf{q}_1 + a_2(t)\mathbf{q}_2 + \ldots,
$$
with $\mathbf{q}_0$, $\mathbf{q}_1$, $\mathbf{q}_2$ orthogonal. Plot the shift mode. Comment on your choice of modal decomposition routine. 

Hint: This can be achieved by performing a modal decomposition technique on $\mathbf{q}-\mathbf{q}_{\textrm{FP}}$ where $\mathbf{q}$ is sampled from flow data on the limit cycle.
(10 marks)

With our three-mode basis, we can now construct the timeseries data for our SINDy model. This timeseries data should consist of the amplitudes of these POD modes for data that represents the growth and saturation of the instability.

**Problem 4**: Project data showcasing the growth and saturation of the limit cycle onto the time-dependent variables $a_1$, $a_2$, $a_0$, where $a_1$, $a_2$ represents the limit cycle, and $a_0$ is a slow variable that represents the shift from the unstable fixed point to the mean-flow. Visualise this reduced description. (5 marks)

**Problem 5** Use SINDy to find a model for the evolution of $a_1$, $a_2$, and $a_0$. Adjust the threshold until the model contains only terms up to and including quadratic interactions. Run your model starting from an initial condition near the mean-flow and visualise the results. Can your model be interpreted? (10 marks)